In [1]:
cd /Users/martin/Git/estates

/Users/martin/Git/estates


In [2]:
import os
import boto3
import json
from decimal import Decimal
import pandas as pd

In [3]:
file = 'data/bronze/20210703-104535_1038932572.json'

In [4]:
with open(file) as f:
    data = json.load(f)

In [6]:
columns = [
    'estate_id',
    'created_at',
    'expires_at',
    'estate_title',
    'estate_description_short',
    'estate_description_long',
    'estate_locality_district',
    'estate_disposition',
    'estate_category_main_cb',
    'estate_longitude',
    'estate_latitude',
    'estate_map_zoom',
    'estate_images',
    'seller_ico',
    'seller_email',
    'seller_numbers',
    'seller_web',
    'seller_address',
    'seller_name',
    'Celková cena',
    'Užitná plocha',
    'Podlaží',
    'Stavba',
    'Stav objektu',
    'Poznámka k ceně',
    'Energetická náročnost budovy',
    'Vlastnictví',
    'Vybavení',
    'Výtah',
]

In [7]:
data

{'1038932572': {'estate_title': 'Prodej  rodinného domu 382\xa0m², pozemek 898\xa0m²',
  'estate_description_short': 'Rodinný dům 382 m² k prodeji Černé Voděrady, okres Praha-východ; 10\xa0985\xa0000 Kč, garáž, patrový, samostatný, smíšená stavba, v dobrém stavu.',
  'estate_description_long': 'S plným právním servisem pro Vás zařídím prodej velkého rodinného domu na okraji v Národní přírodní rezervace Voděradské bučiny v Černých Voděradech. Bučiny jsou nejrozsáhlejším lesním porostem v okolí hlavního města (653 ha).<br /><br />   Dům zkolaudovaný v osmdesátých letech je koncipován jako dvougenerační s dispozicí 2 x 3+1. V přízemí, které je částečně pod úrovní terénu, jsou umístěny technické prostory - dvougaráž, kotelna, prádelna, technický sklad, sklep na zeleninu ... Konstrukce domu je zděná, stropy jsou z keramických nosníků a vložek. Dům je v současné době vyklizený a připravený k prodeji.<br /><br />   Celková plocha pozemku je 898 m2 z  toho zastavěná plocha tvoří 110 m2. 788 m2

In [7]:
data = {
    _id: {item: value for item, value in items.items() if item in columns}
    for _id, items in data.items()
}
ddb_data = json.loads(json.dumps(data), parse_float=Decimal)

In [12]:
dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('estates')

with table.batch_writer() as batch:
    for _id, items in ddb_data.items():
        batch.put_item(
            Item={
                'estate_id': _id,
                'items': items
            }
        )